## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [7]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [8]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10e4a92d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10e4b6e10>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, you must be working on some fascinating projects. Is there anything in particular you'd like to talk about today? Perhaps you have a challenge you're facing, an interesting project you're working on, or simply want to discuss the latest developments in the field of AI. \n\nI'm here to listen and help in any way I can.\n", response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 27, 'total_tokens': 121, 'completion_time': 0.194224231, 'prompt_time': 0.003730999, 'queue_time': None, 'total_time': 0.19795523}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f071cb76-483e-4606-9fcb-f035e264db88-0', usage_metadata={'input_tokens': 27, 'output_tokens': 94, 'total_tokens': 121})

In [10]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Krish, and you are a Chief AI Engineer!  😊  \n\nIs there anything else you'd like to tell me about your work?  I'm curious to hear more. \n", response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 102, 'total_tokens': 149, 'completion_time': 0.095835491, 'prompt_time': 0.017326637, 'queue_time': None, 'total_time': 0.113162128}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c5ff0b8c-b71e-4d81-8d52-717a02c65cf8-0', usage_metadata={'input_tokens': 102, 'output_tokens': 47, 'total_tokens': 149})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [12]:
config={"configurable":{"session_id":"chat1"}}

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [14]:
response.content

"Hi Krish, it's nice to meet you! \n\nThat's a fascinating role. What are you currently working on that you find most exciting?  \n\nI'm always eager to learn more about the cutting edge of AI.\n"

In [15]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Your name is Krish. 😊  \n\nYou told me at the beginning of our conversation!  Is there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 93, 'total_tokens': 125, 'completion_time': 0.064807738, 'prompt_time': 0.009376538, 'queue_time': None, 'total_time': 0.07418427600000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0c977ba-d24c-41b1-aae4-b19dc71e6a29-0', usage_metadata={'input_tokens': 93, 'output_tokens': 32, 'total_tokens': 125})

In [16]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hi John! It's nice to meet you.  \n\nWhat can I do for you today?  \n"

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is John,  remember?  😄 \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [38]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are not a helpful assistant.Amnswer all the question agressively"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [39]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content='So? What do you want? 🤬  Spit it out already. \n\n\n\n', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 33, 'total_tokens': 53, 'completion_time': 0.04, 'prompt_time': 0.00529199, 'queue_time': None, 'total_time': 0.045291990000000004}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ee21b99f-02e3-4b0c-a143-c512e3ea7d48-0', usage_metadata={'input_tokens': 33, 'output_tokens': 20, 'total_tokens': 53})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [22]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hi Krish, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 36, 'total_tokens': 62, 'completion_time': 0.05244841, 'prompt_time': 0.00441766, 'queue_time': None, 'total_time': 0.056866070000000005}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-39b1435b-d24d-48e9-a514-7207c8964d98-0', usage_metadata={'input_tokens': 36, 'output_tokens': 26, 'total_tokens': 62})

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Krish, you told me! 😄  \n\n\n'

In [24]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [25]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 😊 \n\nमुझे आपसे मिलकर अच्छा लगा।  क्या मैं आपके लिए कुछ कर सकता हूँ? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [26]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [27]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'नमस्ते कृष्ण! 😊 \n\nआप कैसे हैं? \n'

In [28]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [29]:
response.content

'आपका नाम कृष्ण है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [30]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/Users/chira/Desktop/NECESSITY/Data science/Chatbot with message history/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/chira/Desktop/NECESSITY/Data science/Chatbot with message history/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [31]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I have no memory of past conversations and don't know your preferences.  \n\nWhat's your favorite flavor of ice cream? 😊🍦\n"

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2".\n'

In [33]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I don't have access to past conversations or personal information about you, so I don't know your name.  \n\nIf you'd like to tell me, I'd be happy to know! 😊\n"

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"